IN THIS CODE WE ARE GOING TO PREPROCESS OUR ORIGINAL DATABASE, THE CORD-19 CHALLENGE ONE, SO IT CAN BE EASIER PROCESSED LATER FOR OUT OBJECTIVE. 

We are goiing to develop this pre-process in 3: 
1. Cleanser
2. Eliminate duplicates
3. Select English languae

And as an extra , we are going to write down our personalized STOPWORDS for later. 

*********************



***************


**STEP 1: CLEANSER**

Before developping our code we are going to clean it. Therefore, we are going to use this previous notebook: https://www.kaggle.com/xhlulu/cord-19-eda-parse-json-and-generate-clean-csv/data?
We are almost using the same things that where developed, but with sigle differences. As we want to focus on the BIORXIV part and on the abstracts.


1. IMPORT LIBRARIES

In [ ]:
import os
import json
from pprint import pprint
from copy import deepcopy

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

2. CREATE FUNCTIONS


In [ ]:
def format_name(author):
    middle_name = " ".join(author['middle'])
    
    if author['middle']:
        return " ".join([author['first'], middle_name, author['last']])
    else:
        return " ".join([author['first'], author['last']])


def format_affiliation(affiliation):
    text = []
    location = affiliation.get('location')
    if location:
        text.extend(list(affiliation['location'].values()))
    
    institution = affiliation.get('institution')
    if institution:
        text = [institution] + text
    return ", ".join(text)

def format_authors(authors, with_affiliation=False):
    name_ls = []
    
    for author in authors:
        name = format_name(author)
        if with_affiliation:
            affiliation = format_affiliation(author['affiliation'])
            if affiliation:
                name_ls.append(f"{name} ({affiliation})")
            else:
                name_ls.append(name)
        else:
            name_ls.append(name)
    
    return ", ".join(name_ls)
def format_body(body_text):
    texts = [(di['section'], di['text']) for di in body_text]
    texts_di = {di['section']: "" for di in body_text}
    
    for section, text in texts:
        texts_di[section] += text

    body = ""

    for section, text in texts_di.items():
        body += section
        body += "\n\n"
        body += text
        body += "\n\n"
    
    return body

def format_bib(bibs):
    if type(bibs) == dict:
        bibs = list(bibs.values())
    bibs = deepcopy(bibs)
    formatted = []
    
    for bib in bibs:
        bib['authors'] = format_authors(
            bib['authors'], 
            with_affiliation=False
        )
        formatted_ls = [str(bib[k]) for k in ['title', 'authors', 'venue', 'year']]
        formatted.append(", ".join(formatted_ls))
    return "; ".join(formatted)

In [ ]:
def load_files(dirname):
    filenames = os.listdir(dirname)
    raw_files = []

    for filename in tqdm(filenames):
        filename = dirname + filename
        file = json.load(open(filename, 'rb'))
        raw_files.append(file)
    
    return raw_files

def generate_clean_df(all_files):
    cleaned_files = []
    
    for file in tqdm(all_files):
        features = [
            file['paper_id'],
            file['metadata']['title'],
            format_authors(file['metadata']['authors']),
            format_authors(file['metadata']['authors'], 
                           with_affiliation=True),
            format_body(file['abstract']),
            format_body(file['body_text']),
            format_bib(file['bib_entries']),
            file['metadata']['authors'],
            file['bib_entries']
        ]

        cleaned_files.append(features)

    col_names = ['paper_id', 'title', 'authors',
                 'affiliations', 'abstract', 'text', 
                 'bibliography','raw_authors','raw_bibliography']
    clean_df = pd.DataFrame(cleaned_files, columns=col_names)
    clean_df.head()
    
    return clean_df

**Biorxiv: Exploration**

> -> load all of the json files into a list of nested dictionaries (each dict is an article).

In [ ]:
biorxiv_dir = '/kaggle/input/cord1933k/biorxiv_medrxiv/biorxiv_medrxiv/'
filenames = os.listdir(biorxiv_dir)
print("Number of articles retrieved from biorxiv:", len(filenames))

In [ ]:
all_files = []

for filename in filenames:
    filename = biorxiv_dir + filename
    file = json.load(open(filename, 'rb'))
    all_files.append(file)

In [ ]:
file = all_files[0]
print("Dictionary keys:", file.keys())

->* Abstract*
The abstract dictionary is fairly simple:

In [ ]:
pprint(file['abstract'])

-> eventually, we can look at other elements, as for example the body text


In [ ]:
#dictionary 
print("body_text type:", type(file['body_text']))
print("body_text length:", len(file['body_text']))
print("body_text keys:", file['body_text'][0].keys())

The content: the body text is separated into a list of small subsections, each containing a section and a text key. Since multiple subsection can have the same section, we need to first group each subsection before concatenating everything.

In [ ]:
print("body_text content:")
pprint(file['body_text'][:2], depth=3)

In [ ]:
#let's see what the grouped section titles are for the example above:

texts = [(di['section'], di['text']) for di in file['body_text']]
texts_di = {di['section']: "" for di in file['body_text']}
for section, text in texts:
    texts_di[section] += text

pprint(list(texts_di.keys()))

-> or the metadata

In [ ]:
#dictionary
print(all_files[0]['metadata'].keys())


In [ ]:
print(all_files[0]['metadata']['title'])

In [ ]:
authors = all_files[0]['metadata']['authors']
pprint(authors[:3])

In [ ]:
#use of format name and affiliation
for author in authors:
    print("Name:", format_name(author))
    print("Affiliation:", format_affiliation(author['affiliation']))
    print()

In [ ]:
#another example
#pprint(all_files[4]['metadata'], depth=4)

-> last element, the bibliography

In [ ]:
bibs = list(file['bib_entries'].values())
pprint(bibs[:2], depth=4)

In [ ]:
#format_authors(bibs[1]['authors'], with_affiliation=False)

The following function let you format the bibliography all at once. It only extracts the title, authors, venue, year, and separate each entry of the bibliography with a ;.

In [ ]:
bib_formatted = format_bib(bibs[:5])
print(bib_formatted)

Biorxiv: Generate CSV

In this section, I show you how to manually generate the CSV files.
As you can see, it's now super simple because of the format_ helper functions.
In the next sections, I show you have to generate them in 3 lines using the load_files and generate_clean_dr helper functions.

In [ ]:
cleaned_files = []

for file in tqdm(all_files):
    features = [
        file['paper_id'],
        file['metadata']['title'],
        format_authors(file['metadata']['authors']),
        format_authors(file['metadata']['authors'], 
                       with_affiliation=True),
        format_body(file['abstract']),
        format_body(file['body_text']),
        format_bib(file['bib_entries']),
        file['metadata']['authors'],
        file['bib_entries']
    ]
    
    cleaned_files.append(features)

In [ ]:

col_names = [
    'paper_id', 
    'title', 
    'authors',
    'affiliations', 
    'abstract', 
    'text', 
    'bibliography',
    'raw_authors',
    'raw_bibliography'
]

clean_df = pd.DataFrame(cleaned_files, columns=col_names)
clean_df.head()

In [ ]:
clean_df.to_csv('biorxiv_clean.csv', index=False)


-> we cab do tge same for Generate Custom (PMC), Commercial, Non-commercial licenses, it will be the blow code repeated for each of them (x3)

In [ ]:
#pmc_dir = '/kaggle/input/CORD-19-research-challenge/custom_license/custom_license/pdf_json/'
#pmc_files = load_files(pmc_dir)
#pmc_df = generate_clean_df(pmc_files)
#pmc_df.to_csv('clean_pmc.csv', index=False)
#pmc_df.head()

BUT WE ARE NOT INTERESTED

Now the data is clean , we have decided we are going to focus on the ABSTRACT from the BIORXIV part, and that we'll do. 

In [ ]:
biorxiv_clean = pd.read_csv('/kaggle/input/cord1933k/biorxiv_clean.csv')

In [ ]:
#verification everything is allright
#biorxiv_clean.head(2)
#first paper example
#biorxiv_clean.text[0]

*****************


**STEP 2: ELIMINATE DUPLICATES**

Now we have the CSV datafram, we are going to analize the content.For that , we use a counter.  This part is highly inspired on this previous notebook,  https://www.kaggle.com/maksimeren/covid-19-literature-clustering from which we extracted what interested us. 

-> Adding word count columns for both abstract and body_text can be useful parameters later:


In [ ]:
df_covid['abstract_word_count'] = biorxiv_clean['abstract'].apply(lambda x: len(x.strip().split()))  # word count in abstract
df_covid['body_word_count'] = biorxiv_clean['body_text'].apply(lambda x: len(x.strip().split()))  # word count in body
df_covid['body_unique_words']=biorxiv_clean['body_text'].apply(lambda x:len(set(str(x).split())))  # number of unique words in body
df_covid.head()

In [ ]:
df_covid.info()

In [ ]:
df_covid['abstract'].describe(include='all')

-> When we look at the unique values above, we can see that tehre are duplicates. It may have caused because of author submiting the article to multiple journals. Let's remove the duplicats from our dataset:

(Thank you Desmond Yeoh for recommending the below approach on Kaggle)

In [ ]:
df_covid.drop_duplicates(['abstract', 'body_text'], inplace=True)
df_covid['abstract'].describe(include='all')

In [ ]:
df_covid['body_text'].describe(include='all')

****


**STEP 3: SELECT ENGLISH**

-> Handling multiple languages
Next we are going to determine the language of each paper in the dataframe. Not all of the sources are English and the language needs to be identified so that we know how handle these instances


In [ ]:
from tqdm import tqdm
from langdetect import detect
from langdetect import DetectorFactory

# set seed
DetectorFactory.seed = 0

# hold label - language
languages = []

# go through each text  #CAMBIAR ESTO. 
for ii in tqdm(range(0,len(df))):
    # split by space into list, take the first x intex, join with space
    text = df.iloc[ii]['body_text'].split(" ")
    
    lang = "en"
    try:
        if len(text) > 50: 
            lang = detect(" ".join(text[:50]))
        elif len(text) > 0:
            lang = detect(" ".join(text[:len(text)]))
    # ught... beginning of the document was not in a good format
    except Exception as e:
        all_words = set(text)
        try:
            lang = detect(" ".join(all_words))
        # what!! :( let's see if we can find any text in abstract...
        except Exception as e:
            
            try:
                # let's try to label it through the abstract then
                lang = detect(df.iloc[ii]['abstract_summary'])
            except Exception as e:
                lang = "unknown"
                pass
    
    # get the language    
    languages.append(lang)

In [ ]:
from pprint import pprint

languages_dict = {}
for lang in set(languages):
    languages_dict[lang] = languages.count(lang)
    
print("Total: {}\n".format(len(languages)))
pprint(languages_dict)

-> Lets take a look at the language distribution in the dataset

In [ ]:
df['language'] = languages
plt.bar(range(len(languages_dict)), list(languages_dict.values()), align='center')
plt.xticks(range(len(languages_dict)), list(languages_dict.keys()))
plt.title("Distribution of Languages in Dataset")
plt.show()

We will be dropping any language that is not English.

In [ ]:
df = df[df['language'] == 'en'] 
df.info()

**********************************

**EXTRA: OUR PERSONALIZED STOPWORDS  (TO BE USED  IN THE PROCESSING PART)**

Stopwords can be extracted from various libraries. Dos not matter which one we choose to use, as far as it is the english ones. 

We also need to customize it a bit so they are the most accurate they can be. 

This part was inspired in various notebooks such as: 

https://www.kaggle.com/mobassir/mining-covid-19-scientific-papers 
https://www.kaggle.com/imdevskp/covid-19-analysis-visualization-comparisons
https://www.kaggle.com/imdevskp/covid-19-analysis-visualization-comparisons


**-> OPTION 1:** from wordcloud. 

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

stop_words = set(STOPWORDS)
#https://www.kaggle.com/gpreda/cord-19-solution-toolbox


In [ ]:
print(stop_words)

but we cannot customize them!!! Therefore:

**-> OPTION 2:** from NLTK

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

stop_words.extend(['background', 'methods', 'introduction', 'conclusions', 'results', 
                   'purpose', 'materials', 'discussions','methodology','result analysis',
                   'doi', 'preprint', 'copyright', 'peer', 'reviewed', 'org', 'https', 'et', 'al', 'author', 'figure', 
                   'rights', 'reserved', 'permission', 'used', 'using', 'biorxiv', 'medrxiv', 'license', 'fig', 'fig.', 
                   'al.', 'Elsevier', 'PMC', 'CZI', 'www'])

In [ ]:
print (stop_words)

First visualization of words of the BIORXIV papers so we can have a first idea of what we may find -> WordCloud

In [ ]:
def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stop_words,
        max_words=200,
        max_font_size=30, 
        scale=5,
        random_state=1
    ).generate(str(data))

    fig = plt.figure(1, figsize=(10,10))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=14)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

In [ ]:
show_wordcloud(biorxiv_clean['abstract'], title = 'biorxiv_clean - papers Abstract - frequent words (400 sample)')